stuff는 질문을 query하여 관련있는 지문을 먼저 찾아서 찾은 지문 전체를 prompt로 넣고 llm을 돌린다.

map reduce는 질문을 기반으로 retriever를 통해 찾은 질문과 연관된 지문 단락을 단락 수 만큼 for 문 수행하면서 

각 단락 기준으로 llm 의 결과를 저장함.

각각 저장된 llm의 결과를 prompt로 합쳐서 llm 이 최종 결과를 만들어 냄.


In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore('./.cache/')

# Token 방식으로 텍스트를 인식
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",                 # 특정 기준으로 분할
    chunk_size=600,
    chunk_overlap=100,
)

# UnstruturedFileLoader : 많은 파일들을 불러올 수 있음. docx, xlsx, pdf, ppt, txt 등 많은 파일들을 불러올 수 있는 Loader
loader= UnstructuredFileLoader("./files/대입합불정리_세특포함_디랩.pdf")

docs = loader.load_and_split(text_splitter=splitter)                                                                                                                                           


embeddings = OpenAIEmbeddings()

cacehd_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

# 캐시를 적용하여 임베딩
# vectorstore = Chroma.from_documents(docs, cacehd_embeddings)
vectorstore = FAISS.from_documents(docs, cacehd_embeddings)

retriver = vectorstore.as_retriever()

########### MAP REDUCE ###########

map_doc_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    다음의 긴 Document의 일부 중, 질문에 대한 답변을 생성하는 것과 관련이 있는 부분을 찾아주세요.
    관련이 있는 부분을 찾는다면, 해당 Text 를 그대로 반환해주세요.
    ---
    {context}
    """),
    ("human", "{question}")
])

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    documents = inputs['documents']
    question = inputs['question']

    return "\n\n".join(map_doc_chain.invoke({
        "context": doc.page_content,
        "question": question,
    }).content for doc in documents)

map_chain = {"documents": retriver, "question": RunnablePassthrough()} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    주어진 긴 document의 발췌문들과 question을 통해, 최종 답변을 생성하세요. 
    ---
    {context}
    """),
    ("human", "{question}")
])

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm


chain.invoke("성균관대를 합격자의 평균 내신등급은 어느 정도야 ?")

AIMessage(content='성균관대를 합격한 학생들의 평균 내신은 2.63과 3.10입니다.')